# OUTLIER DETECTION USING GMM AND SVDD

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from data_processing.subject_selectors import *
from data_processing.window_splitters import *
from data_processing.train_test_spliters import *
import pathlib
from os.path import join, isfile
from os import listdir
import itertools
from sklearn.mixture import GaussianMixture
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import json
import warnings 

In [ ]:
df = pd.read_csv(join(pathlib.Path().resolve(), "results/SVM_new/result_1vsN_mfcc_3h.csv"))


In [ ]:
df.iloc[:, df.columns != "params"]

In [ ]:
means = df.iloc[:, df.columns != "params"].mean(axis = 1)
stds = df.iloc[:, df.columns != "params"].std(axis = 1)
newdf = DataFrame()
newdf["params"] = df["params"]
newdf["mean"] = means
newdf["std"] = stds
newdf= newdf.sort_values(by = ["mean"], ascending = False)

In [ ]:
newdf

In [ ]:
# newdf.to_csv("results/SVM_new/final_1vsN_mfcc_3h.csv", index=False)

In [ ]:
newdf = pd.read_csv(join(pathlib.Path().resolve(), "final_2v2_hour.csv"))
newdf= newdf.sort_values(by = ["mean"], ascending = False)
newdf.to_csv("final_2v2_hour.csv", index=False)

In [ ]:
newdf = pd.read_csv(join(pathlib.Path().resolve(), "results/GMM_new/final_1vsN_mfcc_3h.csv"))
newdf

In [ ]:
s = "\\hline \n \
    {} & {} & {}  & {} & {} &{:.3f} & {:.3f} \\\\"

In [ ]:
import json
count = 0
for i, row in newdf.iterrows():
    pos = count + 1
    # print (row["params"].replace("'", "\""))

    aux = json.loads(row["params"].replace("'", "\"").replace("[", "[\"").replace("]", "\"]"))
    # print(aux["trans__pca_components"])
    print(s.format(str(pos) + ".", aux["gmm__n_components"], aux["trans__mfcc_components"], aux["trans__pca_components"], aux["trans__transformations"], row["mean"], row["std"]))
    count += 1
    if count >= 20:
        break

In [ ]:
import json
count = 0
for i, row in newdf.iterrows():
    pos = count + 1

    aux = json.loads(row["params"].replace("'", "\"").replace("[", "[\"").replace("]", "\"]"))

    if aux["gmm__n_components"] == 4 and aux["trans__mfcc_components"] == 5 and aux["trans__pca_components"] == 0 and aux["trans__transformations"] == [""]:
        print(row["mean"])
        print(aux["trans__transformations"])
        break


In [ ]:
x1 = 0.9358780792879168
x2 = 0.7459314715923672 (no MFCC)
x3 = 0.890218025645096 (no stat)

plt.figure(figsize=(3, 5))
# plt.scatter([x1,x2,x3], [0,0,0] )
plt.plot([0, 1], [x1, x2], color='r', marker='o', label = "")
plt.plot([0, 1], [x1, x3], color='g', marker='o')
plt.ylabel("mean AUC")
plt.xticks([])
plt.show()

In [ ]:
import json
count = 0
for i, row in newdf.iterrows():
    pos = count + 1
    aux = json.loads(row["params"].replace("'", "\""))
    print(s.format(str(pos) + ".", aux["gmm__n_components"], aux["pca__n_components"], row["mean"], row["std"]))
    count += 1
    if count >= 20:
        break

In [ ]:
target_row = df[df["params"] == "{'gmm__n_components': 32, 'pca__n_components': 500}"]
target_row = target_row.to_numpy()[:, 1:].flatten().astype(float)
print(type(target_row))
print(target_row.shape)
target_row = target_row[~np.isnan(target_row)]
target_row

In [ ]:
x_axis = np.arange(1, 101)

counts, bins = np.histogram(target_row)
print(counts)
print(bins)
plt.hist(target_row)
plt.xlabel("AUC")
plt.ylabel("Number of combinations")
plt.show()

In [ ]:
df1 = pd.read_csv(join(pathlib.Path().resolve(), "results/GMM/final_1vsN_moreDist.csv"))
df2 = pd.read_csv(join(pathlib.Path().resolve(), "results/GMM/final_1vsN_moreDist2.csv"))
df3 = pd.read_csv(join(pathlib.Path().resolve(), "results/GMM/final_1vsN.csv"))

In [ ]:
aux = pd.concat([df1, df2, df3])
aux = aux.sort_values(by = ["mean"], ascending = False)
aux.to_csv("results/GMM/final_1vsN_all.csv", index=False)

In [ ]:
res1 = None
res2 = None

with open(join(pathlib.Path().resolve(), "results/GMM_new/result_NvsN_mfcc_one_3h.json"), "r") as f:
    res1 = json.load(f)

with open(join(pathlib.Path().resolve(), "results/GMM_new/result_NvsN_mfcc_multiple_3h.json"), "r") as f:
    res2 = json.load(f)

print(res1)
print(res2)


In [ ]:
x1 = []
x2 = []
for key in res1.keys():
    x1.append([int(key), np.mean(res1[key]), np.std(res1[key])])

for key in res2.keys():
    x2.append([int(key), np.mean(res2[key]), np.std(res2[key])])

x1 = np.array(x1)
x2 = np.array(x2)
print(x1[:, 2])
print(x2[:, 2])
# print(x2)

In [ ]:



colors = ["blue", "blue", "blue", "blue", "blue", "red", "red", "red", "red", "red"]


plt.errorbar(x1[:, 0], x1[:, 1], x1[:, 2], c = "blue", label = "One model", linestyle='None', marker="o", capsize = 5)
plt.errorbar(x2[:, 0], x2[:, 1], x2[:, 2], c = "red", label = "Multiple models", linestyle='None', marker="o", capsize=5)

plt.xlabel("Number of authorised subjects")
plt.xticks([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
plt.ylabel("AUC")
plt.legend()
plt.show()

In [ ]:
x = np.array([1, 2, 3, 4, 5])
y = np.power(x, 2) # Effectively y = x**2
e = np.array([1.5, 2.6, 3.7, 4.6, 5.5])

plt.errorbar(x, y, e, linestyle='None', marker='^')

plt.show()